# Model Evaluation and Testing — From Development to Production

**Audience:** ML developers & data scientists  
**Scope:** Metrics, testing methodology, offline/online evaluation, monitoring & drift  
**Includes:** Explanations, formulas (LaTeX), and runnable Python code

---

## Table of Contents
1. Setup
2. Datasets & Reproducibility
3. Data Quality & Split Validation
4. Classification Metrics (formulas + code)
5. Regression Metrics (formulas + code)
6. Calibration & Thresholding
7. Curves & Diagnostic Plots (ROC/PR, Confusion Matrix)
8. Clustering Metrics (formulas + code)
9. Cross-Validation & Hyperparameter Tuning
10. Robustness, Bias & Fairness (practical checks)
11. Performance, Latency & Throughput (quick benchmarks)
12. Drift & Monitoring (with formulas + code sketch)
13. Testing in MLOps (unit, integration, regression tests)
14. Developer Checklists & Tips

## 1. Setup

In [ ]:
!pip -q install scikit-learn matplotlib numpy pandas evaluate shap fairlearn

In [ ]:
import warnings, os, math, time, sys, itertools
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, ConfusionMatrixDisplay, roc_curve, precision_recall_curve,
    average_precision_score, mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, load_diabetes, load_breast_cancer

np.random.seed(42)

## 2. Datasets & Reproducibility

In [ ]:
# Classification
X_cls, y_cls = load_breast_cancer(return_X_y=True, as_frame=True)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, stratify=y_cls, random_state=42
)

# Regression
X_reg, y_reg = load_diabetes(return_X_y=True, as_frame=True)
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Clustering
from sklearn.datasets import make_blobs
X_clu, y_clu_true = make_blobs(n_samples=600, centers=4, cluster_std=1.2, random_state=42)

(X_cls.head(), X_reg.head(), X_clu[:3])

## 3. Data Quality & Split Validation

In [ ]:
def quick_profile(df: pd.DataFrame, name: str):
    print(f"--- {name} shape:", df.shape)
    print("Missing values per column:\n", df.isna().sum().sort_values(ascending=False)[:5])
    print("\nDescribe:\n", df.describe().T.head())

quick_profile(Xc_train, "Xc_train")
quick_profile(Xc_test, "Xc_test")

In [ ]:
from scipy.stats import ks_2samp

def ks_split_check(train_df, test_df, top_n=5):
    stats = []
    for col in train_df.columns:
        stat, p = ks_2samp(train_df[col].values, test_df[col].values)
        stats.append((col, stat, p))
    stats = sorted(stats, key=lambda x: -x[1])[:top_n]
    print("Top features by KS statistic (train vs test):")
    for col, stat, p in stats:
        print(f"{col:25s} KS={stat:.3f}  p={p:.3f}")

ks_split_check(Xc_train, Xc_test)

## 4. Classification Metrics — Formulas & Code

Let $TP$, $TN$, $FP$, $FN$ denote confusion matrix components.

- **Accuracy:** $\\frac{TP + TN}{TP + TN + FP + FN}$  
- **Precision:** $\\frac{TP}{TP + FP}$  
- **Recall:** $\\frac{TP}{TP + FN}$  
- **F1:** $2\\cdot\\frac{PR}{P+R}$ with $P$ = Precision, $R$ = Recall  
- **ROC AUC:** area under ROC; $TPR=\\frac{TP}{TP+FN}$, $FPR=\\frac{FP}{FP+TN}$

In [ ]:
pipe_cls = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500, solver="lbfgs"))
pipe_cls.fit(Xc_train, yc_train)

y_pred = pipe_cls.predict(Xc_test)
y_proba = pipe_cls.predict_proba(Xc_test)[:,1]

acc = accuracy_score(yc_test, y_pred)
prec = precision_score(yc_test, y_pred)
rec = recall_score(yc_test, y_pred)
f1 = f1_score(yc_test, y_pred)
auc = roc_auc_score(yc_test, y_proba)

print(f"Accuracy: {acc:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f} | ROC-AUC: {auc:.3f}")

## 5. Regression Metrics — Formulas & Code

- **MAE:** $\\frac{1}{n}\\sum |y-\\hat{y}|$  
- **MSE:** $\\frac{1}{n}\\sum (y-\\hat{y})^2$  
- **RMSE:** $\\sqrt{\\mathrm{MSE}}$  
- **$R^2$:** $1-\\frac{\\sum (y-\\hat{y})^2}{\\sum (y-\\bar{y})^2}$

In [ ]:
pipe_reg = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=200, random_state=42))
pipe_reg.fit(Xr_train, yr_train)
yhat = pipe_reg.predict(Xr_test)

import math
mae = mean_absolute_error(yr_test, yhat)
mse = mean_squared_error(yr_test, yhat)
rmse = math.sqrt(mse)
r2 = r2_score(yr_test, yhat)

print(f"MAE: {mae:.3f} | MSE: {mse:.3f} | RMSE: {rmse:.3f} | R^2: {r2:.3f}")

## 6. Calibration & Thresholding

In [ ]:
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(yc_test, y_proba, n_bins=10, strategy="uniform")

plt.figure(figsize=(6,4))
plt.plot([0,1],[0,1], linestyle='--')
plt.plot(prob_pred, prob_true, marker='o')
plt.xlabel("Predicted probability")
plt.ylabel("Observed frequency")
plt.title("Reliability Diagram")
plt.grid(True); plt.show()

In [ ]:
# Threshold tuning for best F1
ths = np.linspace(0, 1, 101)
f1s = []
for t in ths:
    y_hat = (y_proba >= t).astype(int)
    f1s.append(f1_score(yc_test, y_hat))

best_t = ths[int(np.argmax(f1s))]
print("Best threshold by F1:", best_t, "F1:", max(f1s))

## 7. Curves & Diagnostic Plots (ROC/PR, Confusion Matrix)

In [ ]:
# ROC
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score, ConfusionMatrixDisplay

fpr, tpr, _ = roc_curve(yc_test, y_proba)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1], linestyle='--')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve"); plt.grid(True); plt.show()

In [ ]:
# Precision-Recall
precisions, recalls, _ = precision_recall_curve(yc_test, y_proba)
ap = average_precision_score(yc_test, y_proba)

plt.figure(figsize=(6,4))
plt.step(recalls, precisions, where='post')
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"Precision-Recall Curve (AP={ap:.3f})")
plt.grid(True); plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(yc_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative","Positive"])
disp.plot(values_format='d'); plt.title("Confusion Matrix"); plt.show()

## 8. Clustering Metrics — Formulas & Code

Silhouette for sample $i$: $s(i)=\\frac{b(i)-a(i)}{\\max(a(i),b(i))}$,  
where $a(i)$ = mean intra-cluster distance, $b(i)$ = mean distance to nearest other cluster.

DBI (lower better) and CHI (higher better) provide global clustering quality.

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_clu)

sil = silhouette_score(X_clu, labels)
dbi = davies_bouldin_score(X_clu, labels)
chi = calinski_harabasz_score(X_clu, labels)

print(f"Silhouette: {sil:.3f} | DBI (lower better): {dbi:.3f} | CHI (higher better): {chi:.1f}")

## 9. Cross-Validation & Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500))
scores = cross_val_score(clf, X_cls, y_cls, cv=cv, scoring="f1")
print("5-fold CV F1:", scores, "Mean:", scores.mean())

In [ ]:
param_grid = {"logisticregression__C":[0.01, 0.1, 1, 10]}
gs = GridSearchCV(
    make_pipeline(StandardScaler(), LogisticRegression(max_iter=500)),
    param_grid=param_grid, scoring="f1", cv=cv, n_jobs=-1
)
gs.fit(Xc_train, yc_train)
print("Best params:", gs.best_params_, "Best CV F1:", gs.best_score_)

## 10. Robustness, Bias & Fairness

In [ ]:
# Simple subgroup performance demo using a synthetic group
group = (Xc_test.iloc[:,0] > Xc_test.iloc[:,0].median()).astype(int).values

def subgroup_metrics(y_true, y_prob, g):
    out = {}
    for val in [0,1]:
        idx = (g==val)
        yhat = (y_prob[idx] >= 0.5).astype(int)
        out[f"group{val}_f1"] = f1_score(y_true[idx], yhat)
        out[f"group{val}_recall"] = recall_score(y_true[idx], yhat)
        out[f"group{val}_precision"] = precision_score(y_true[idx], yhat)
    return out

subgroup_metrics(yc_test.values, y_proba, group)

In [ ]:
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, false_positive_rate

mf = MetricFrame(
    metrics={"selection_rate": selection_rate, "tpr": true_positive_rate, "fpr": false_positive_rate},
    y_true=yc_test, y_pred=(y_proba>=0.5).astype(int), sensitive_features=group
)
print("By-group metrics:\n", mf.by_group)
print("\nOverall disparity (range):")
for name, s in mf.by_group.items():
    print(name, "range:", s.max() - s.min())

## 11. Performance, Latency & Throughput

In [ ]:
X_batch = Xc_test.values[:1024]
start = time.time()
_ = pipe_cls.predict_proba(X_batch)
elapsed = time.time() - start
latency_ms = (elapsed / len(X_batch)) * 1000
tps = len(X_batch) / elapsed
print(f"Avg latency: {latency_ms:.3f} ms/sample | Throughput: {tps:.1f} samples/s")

## 12. Drift & Monitoring

Population Stability Index (PSI): $\\sum_i (o_i-e_i)\\ln\\left(\\frac{o_i}{e_i}\\right)$

In [ ]:
def psi(expected, observed, bins=10):
    e_perc, _ = np.histogram(expected, bins=bins, range=(np.min(expected), np.max(expected)), density=True)
    o_perc, _ = np.histogram(observed, bins=bins, range=(np.min(expected), np.max(expected)), density=True)
    e_perc = np.where(e_perc==0, 1e-6, e_perc)
    o_perc = np.where(o_perc==0, 1e-6, o_perc)
    return np.sum((o_perc - e_perc) * np.log(o_perc / e_perc))

prod_feature = Xc_test.iloc[:,0].values + np.random.normal(0, 0.2, size=len(Xc_test))
psi_val = psi(Xc_train.iloc[:,0].values, prod_feature, bins=10)
print("PSI (feature 0):", psi_val)

## 13. Testing in MLOps

In [ ]:
def assert_not_regressed(old_f1: float, new_f1: float, drop_tol: float = 0.01):
    if new_f1 + drop_tol < old_f1:
        raise AssertionError(f"Model regressed: old_f1={old_f1:.3f}, new_f1={new_f1:.3f} exceeds tolerance {drop_tol}")
    return True

baseline_f1 = 0.95
current_f1 = f1
assert_not_regressed(baseline_f1, current_f1, drop_tol=0.05)
print("Regression test passed within tolerance.")

## 14. Developer Checklists & Tips

- Align metrics to business costs (precision/recall trade-offs).  
- Stratified splits, leakage checks, and data validation.  
- Track metrics & params (MLflow/DVC).  
- Add CI checks: unit, integration, model regression.  
- Monitor drift (PSI/KS) & alerting; recalibrate thresholds after prevalence shifts.  
- Use SHAP/LIME for transparency where required.